In [1]:
!date

Sun Sep 25 10:50:29 CST 2016


# Running recipes in Jupyter notebook

In this notebook we will demostrate how to use the chef module to create a dataset

In [2]:
import os
import pandas as pd
import json
import yaml
import numpy as np

from ddf_utils import chef

import logging
logging.basicConfig(level=logging.DEBUG)

## loading recipes

recipes can be either yaml or json files. Because recipe may include other recipes and translation dictionaries, chef module provides a build_recipe() function to read recipe and expand the recipe with other included files.

In [5]:
print(open("../tests/recipes/recipe_cme.yaml").read())  # Note: you should change the ddf_dir to correct path


info:
    id: cme-sg-dataset
    base:
        - &d1 ddf--gapminder--gapminder_world
        - &d2 ddf--gapminder--geo_entity_domain
        - &d3 ddf--cme

config:
    ddf_dir: /Users/semio/src/work/Gapminder
    recipes_dir: ./
    dictionary_dir: ../translation_dictionaries/
        
include:
    - recipe_gw_common.yaml

ingredients:
    # CME
    - id: cme-datapoints
      dataset: *d3
      key: "country,year"
      value: ['u5mr_median', 'nmr_median', 'neonatal_deaths_median']
    - id: cme-countries
      dataset: *d3
      key:
          - country
      value: "*"

    # GW
    - id: gw-entities
      dataset: *d2
      key: [geo]
      value: "*"
    - id: gw-countries
      dataset: *d2
      key: [country]
      value: "*"
    - id: gw-concepts-cme
      dataset: *d1
      key: concept
      value: "*"
      filter:
          concept:
              - newborn_deaths
              - newborn_mortality_rate_per_1000
              - under_five_mortality_from_cme_per_1000_born

c

In [6]:
recipe = chef.build_recipe("../tests/recipes/recipe_cme.yaml")

In [7]:
from pprint import pprint

pprint(recipe)

{'config': {'ddf_dir': '/Users/semio/src/work/Gapminder',
            'dictionary_dir': '../translation_dictionaries/',
            'recipes_dir': './'},
 'cooking': {'concepts': [{'ingredients': ['gw-concepts-cme',
                                           'gw-concepts-geo',
                                           'gw-concepts-discrete'],
                           'options': AttrDict([('deep', True)]),
                           'procedure': 'merge',
                           'result': 'cme-concepts-final'}],
             'datapoints': [{'ingredients': ['cme-datapoints'],
                             'options': {'dictionary': {'neonatal_deaths_median': 'newborn_deaths',
                                                        'nmr_median': 'newborn_mortality_rate_per_1000',
                                                        'u5mr_median': 'under_five_mortality_from_cme_per_1000_born'}},
                             'procedure': 'translate_header',
                           

## running the recipe

the run_recipe() function from chef module actually run the recipe and return the result DDF as a dictionary of ingredients.

In [8]:
res = chef.run_recipe(recipe)

running concepts
running entities
running datapoints


In [9]:
res.keys()

dict_keys(['entities', 'datapoints', 'concepts'])

### entities

In [12]:
entities = res['entities'].get_data()

In [13]:
entities.keys()  # in the recipe we include all geo entities from GW, so they are.

dict_keys(['main_religion_2008', 'g77_and_oecd_countries', 'world_4region', 'geographic_regions_in_4_colors', 'income_groups', 'geographic_regions', 'global', 'country', 'landlocked'])

In [14]:
entities['main_religion_2008']

,main_religion_2008,name,gwid,is--main_religion_2008
0,muslim,Muslim,i280,TRUE
1,eastern_religions,Eastern religions,i281,TRUE
2,christian,Christian,i279,TRUE


### datapoints

In [15]:
dps = res['datapoints'].get_data()

In [16]:
dps.keys()

dict_keys(['under_five_mortality_from_cme_per_1000_born', 'newborn_deaths', 'newborn_mortality_rate_per_1000'])

In [17]:
dps['under_five_mortality_from_cme_per_1000_born'].head()

,geo,time,under_five_mortality_from_cme_per_1000_born
0,afg,1961,356.5
1,afg,1962,350.6
2,afg,1963,345.0
3,afg,1964,339.7
4,afg,1965,334.1


### concepts

In [19]:
concepts = res['concepts'].get_data()

In [20]:
concepts.keys()

dict_keys(['concept'])

In [22]:
concepts['concept'].concept.values

array(['under_five_mortality_from_cme_per_1000_born',
       'newborn_mortality_rate_per_1000', 'newborn_deaths',
       'geographic_regions', 'income_groups', 'landlocked',
       'g77_and_oecd_countries', 'geographic_regions_in_4_colors',
       'main_religion_2008', 'geo', 'country', 'world_4region', 'latitude',
       'longitude', 'global', 'gapminder_list', 'alternative_1',
       'alternative_2', 'alternative_3', 'alternative_4_cdiac', 'pandg',
       'god_id', 'alt_5', 'upper_case_name', 'code', 'number', 'gwid',
       'arb1', 'arb2', 'arb3', 'arb4', 'arb5', 'arb6', 'arb7', 'domain',
       'scales', 'color', 'drill_up', 'description', 'indicator_url',
       'name', 'unit', 'interpolation', 'name_short', 'name_long', 'age',
       'time'], dtype=object)